In [1]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

In [ ]:


# Create a directory to save the HTML files
directory = "amazon-docs"
if not os.path.exists(directory):
    os.makedirs(directory)

# Base URL of the website
base_url = "https://www.aboutamazon.com/news/workplace/amazons-20-weeks-of-paid-leave-helped-this-new-mom-find-balance"

# Function to download and save HTML content
def save_html(url, path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(path, 'w', encoding='utf-8') as file:
            file.write(response.text)
    else:
        print(f"Failed to retrieve {url}")

# Function to recursively download and save linked pages
def download_linked_pages(url, visited, depth=0):
    if depth > 5:  # Adjust the depth limit as needed
        return
    if url in visited:
        return
    visited.add(url)
    
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        filename = urlparse(url).path.replace('/', '_') + '.html'
        file_path = os.path.join(directory, filename)
        save_html(url, file_path)
        
        # Find all links on the page
        links = soup.find_all('a', href=True)
        for link in links:
            href = link['href']
            # Skip external links
            if urlparse(href).netloc and urlparse(href).netloc != urlparse(base_url).netloc:
                continue
            # Convert relative URLs to absolute URLs
            file_url = urljoin(base_url, href)
            download_linked_pages(file_url, visited, depth + 1)
    else:
        print(f"Failed to retrieve {url}")

# Start the recursive download from the main page
visited_urls = set()
download_linked_pages(base_url, visited_urls)